# Nuclia | Nuclia> [Nuclia](https://nuclia.com)は、内部および外部のあらゆるソースからの非構造化データを自動的にインデックス化し、最適化された検索結果と生成的な回答を提供します。ビデオとオーディオの書き起こし、画像コンテンツの抽出、文書のパースが可能です。>> > [Nuclia](https://nuclia.com) automatically indexes your unstructured data from any internal and external source, providing optimized search results and generative answers. It can handle video and audio transcription, image content extraction, and document parsing.> `Nuclia Understanding API`は、テキスト、ウェブページ、ドキュメント、オーディオ/ビデオコンテンツを含む非構造化データの処理をサポートしています。必要に応じて音声認識やOCRを使用して、どこにあるテキストでも抽出します。また、メタデータ、PDF内の画像のような埋め込まれたファイル、ウェブリンクも抽出します。機械学習が有効になっている場合、エンティティを識別し、コンテンツの要約を提供し、すべての文の埋め込みを生成します。>> > The `Nuclia Understanding API` supports the processing of unstructured data, including text, web pages, documents, and audio/video contents. It extracts all texts wherever they are (using speech-to-text or OCR when needed), it also extracts metadata, embedded files (like images in a PDF), and web links. If machine learning is enabled, it identifies entities, provides a summary of the content and generates embeddings for all the sentences.

## Setup | セットアップ

`Nuclia Understanding API`を使用するには、Nucliaアカウントが必要です。<https://nuclia.cloud>で無料アカウントを作成し、その後[NUAキーを作成する](https://docs.nuclia.dev/docs/docs/using/understanding/intro)ことができます。> To use the `Nuclia Understanding API`, you need to have a Nuclia account. You can create one for free at <https://nuclia.cloud>, and then [create a NUA key](https://docs.nuclia.dev/docs/docs/using/understanding/intro).

In [ ]:
#!pip install --upgrade protobuf
#!pip install nucliadb-protos

In [3]:
import os

os.environ["NUCLIA_ZONE"] = "<YOUR_ZONE>"  # e.g. europe-1
os.environ["NUCLIA_NUA_KEY"] = "<YOUR_API_KEY>"

## Example | 例Nucliaドキュメントローダーを使用するには、`NucliaUnderstandingAPI`ツールをインスタンス化する必要があります。> To use the Nuclia document loader, you need to instantiate a `NucliaUnderstandingAPI` tool:

In [ ]:
from langchain.tools.nuclia import NucliaUnderstandingAPI

nua = NucliaUnderstandingAPI(enable_ml=False)

In [ ]:
from langchain.document_loaders.nuclia import NucliaLoader

loader = NucliaLoader("./interview.mp4", nua)

ドキュメントを取得するまで、ループ内で`load`を呼び出すことができるようになりました。> You can now call the `load` the document in a loop until you get the document.

In [ ]:
import time

pending = True
while pending:
    time.sleep(15)
    docs = loader.load()
    if len(docs) > 0:
        print(docs[0].page_content)
        print(docs[0].metadata)
        pending = False
    else:
        print("waiting...")

## Retrieved information | 取得した情報Nucliaは以下の情報を返します：> Nuclia returns the following information:* ファイルのメタデータ  > file metadata* 抽出されたテキスト  > extracted text* 埋め込まれた画像の中のテキストのようなネストされたテキスト  > nested text (like text in an embedded image)* 段落と文の分割（それぞれの最初と最後の文字の位置によって定義され、ビデオやオーディオファイルの場合は開始時刻と終了時刻も含む）  > paragraphs and sentences splitting (defined by the position of their first and last characters, plus start time and end time for a video or audio file)* リンク  > links* サムネイル  > a thumbnail* 埋め込まれたファイル  > embedded files注意：> Note:生成されたファイル（サムネイル、抽出された埋め込みファイルなど）はトークンとして提供されます。これらは[`/processing/download` エンドポイント](https://docs.nuclia.dev/docs/api#operation/Download_binary_file_processing_download_get)を使用してダウンロードできます。> Generated files (thumbnail, extracted embedded files, etc.) are provided as a token. You can download them with the [`/processing/download` endpoint](https://docs.nuclia.dev/docs/api#operation/Download_binary_file_processing_download_get).また、どのレベルでも、属性が一定のサイズを超えると、それはダウンロード可能なファイルに格納され、ドキュメント内ではファイルポインターに置き換えられます。これは `{"file": {"uri": "JWT_TOKEN"}}` の形式になります。ルールとして、メッセージのサイズが100万文字を超える場合、最も大きな部分がダウンロード可能なファイルに移動されます。まず、圧縮プロセスはベクトルを対象とします。それが不十分な場合は、大きなフィールドのメタデータを対象とし、最終的には抽出されたテキストを対象とします。> Also at any level, if an attribute exceeds a certain size, it will be put in a downloadable file and will be replaced in the document by a file pointer. This will consist of `{"file": {"uri": "JWT_TOKEN"}}`. The rule is that if the size of the message is greater than 1000000 characters, the biggest parts will be moved to downloadable files. First, the compression process will target vectors. If that is not enough, it will target large field metadata, and finally it will target extracted text.